# 라이브러리

In [1]:
import numpy as np
import pandas as pd
import pickle
import os
import geocoder
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
font_path = "/Library/Fonts/Arial Unicode.ttf"
fontprop = fm.FontProperties(fname=font_path, size=18)

# 데이터 불러오기

In [2]:
path = "/Users/heogeonhyeock/Desktop/인빅_프로젝트/교통량"
file_list = os.listdir(path)

for i in range(len(file_list)):
    try:
        globals()["trafic_{}".format(i)] = pd.read_csv(path + "/" + file_list[i],
                                                        encoding='cp949')
    except:
        pass
trafic = pd.concat([trafic_1,trafic_2,trafic_3,trafic_4,trafic_5])
trafic

,집계일자,출발권역명,수도권도착,강원도착,대전충남도착,광주전남도착,대구경북도착,부산경남도착,전북도착,충북도착,Unnamed: 10
0,20210401,수도권본부,537405,56887,72633,3576,8589,6016,7166,52410,NaN
1,20210401,강원본부,57238,97204,1966,511,2067,877,668,15185,NaN
2,20210401,대전충남본부,71187,2196,143868,4520,9162,4823,28094,16653,NaN
3,20210401,광주전남본부,3259,533,4109,173834,4057,14154,20971,1390,NaN
4,20210401,대구경북본부,8044,1978,8412,3942,260308,39287,2250,8262,NaN
...,...,...,...,...,...,...,...,...,...,...,...
243,20210331,광주전남본부,2933,464,3974,173260,3948,12327,20827,1392,NaN
244,20210331,대구경북본부,7577,1814,8260,3847,233399,37800,2213,8039,NaN
245,20210331,부산경남본부,5595,840,3599,12096,34302,455447,3758,1810,NaN
246,20210331,전북본부,6759,598,27586,20706,2080,4033,33833,2691,NaN


In [3]:
with open("/Users/heogeonhyeock/Desktop/인빅_프로젝트/FC주문/FC_order_distance.pickle","rb") as fr:
    FC_order = pickle.load(fr)
FC_order["송화 -> 터미널"] = None
FC_order["터미널 -> 수화"] = None
FC_order = FC_order[["INS_DATE","SHPR_ADDR_1","CNEE_ADDR_1","송화 -> 터미널","터미널 -> 수화"]]
FC_order

,INS_DATE,SHPR_ADDR_1,CNEE_ADDR_1,송화 -> 터미널,터미널 -> 수화
1,20210303,경기도,경상남도,None,None
2,20210303,경기도,경상남도,None,None
3,20210303,경기도,서울특별시,None,None
4,20210303,경기도,경기도,None,None
5,20210303,경기도,경기도,None,None
...,...,...,...,...,...
1786343,20210702,경기도,서울특별시,None,None
1786347,20210702,경기도,경기도,None,None
1786348,20210703,경기도,제주특별자치도,None,None
1786349,20210704,경기도,대전광역시,None,None


# 지역 구분

In [4]:
loc_dict = {"수도권본부" : ["경기도", "서울특별시", "경기도", "인천광역시", "서울", "경기", "인천", "인천시"],
     "강원본부" : ["강원도", "강원"],
     "대전충남본부" : ["충청남도", "대전광역시", "세종특별자치시", "대전", "충남"],
     "광주전남본부" : ["전라남도", "광주광역시", "광주", "전남"],
     "대구경북본부" : ["경상북도", "대구광역시", "경북", "대구", "대구시"],
     "부산경남본부" : ["경상남도", "부산광역시", "울산광역시", "경남", "부산", "울산", "부산시"],
     "전북본부" : ["전라북도", "전북"],
     "충북본부" : ["충청북도", "충북"],
     "권역 외" : ["제주특별자치도", "제주"]}

# 지역별 교통량
송화 - 터미널 : INS_DATE 기준
터미널 - 수화 : INS_DATE 기준 +1

In [5]:
pd.options.mode.chained_assignment = None  # default='warn'
FC_time = np.array(FC_order["INS_DATE"])

for index in FC_order.index:
    for i in list(loc_dict.keys())[:-1]:
        if FC_order["SHPR_ADDR_1"][index] in loc_dict.get(i):
            try:
                FC_order["송화 -> 터미널"][index] = list(trafic[(trafic["집계일자"] == FC_order["INS_DATE"][index]) &\
                                                   (trafic["출발권역명"] == i)]["수도권도착"])[0]
            except:
                print("{}에서 오류".format(index))
                FC_order["송화 -> 터미널"][index] = None
        if FC_order["CNEE_ADDR_1"][index] in loc_dict.get(i):
            try:
                FC_order["터미널 -> 수화"][index] = list(trafic[(trafic["집계일자"] == FC_order["INS_DATE"][index]+1) &\
                                                   (trafic["출발권역명"] == "수도권본부")][i[:-2]+"도착"])[0]
            except:
                print("{}에서 오류".format(index))
                FC_order["터미널 -> 수화"][index] = None
FC_order

294815에서 오류
294816에서 오류
294817에서 오류
294818에서 오류
294819에서 오류
294820에서 오류
294904에서 오류
294905에서 오류
294906에서 오류
294907에서 오류
294908에서 오류
294909에서 오류
294910에서 오류
294911에서 오류
294912에서 오류
294913에서 오류
294914에서 오류
294915에서 오류
294916에서 오류
294917에서 오류
294918에서 오류
294919에서 오류
294990에서 오류
294991에서 오류
294992에서 오류
294993에서 오류
294994에서 오류
294995에서 오류
294996에서 오류
294997에서 오류
294999에서 오류
295000에서 오류
295001에서 오류
295002에서 오류
295003에서 오류
295086에서 오류
295087에서 오류
295088에서 오류
295089에서 오류
295090에서 오류
295091에서 오류
295092에서 오류
295093에서 오류
295094에서 오류
295095에서 오류
295190에서 오류
295246에서 오류
295247에서 오류
295248에서 오류
295249에서 오류
295309에서 오류
295310에서 오류
295343에서 오류
295344에서 오류
295345에서 오류
295346에서 오류
295347에서 오류
295348에서 오류
295349에서 오류
295350에서 오류
295351에서 오류
295352에서 오류
295353에서 오류
295357에서 오류
295376에서 오류
295377에서 오류
295378에서 오류
295412에서 오류
295413에서 오류
295414에서 오류
295415에서 오류
295416에서 오류
295417에서 오류
295418에서 오류
295419에서 오류
295420에서 오류
295421에서 오류
295422에서 오류
295423에서 오류
295424에서 오류
295425에서 오류
295426에서 오류
295445에서 오류
2954

296846에서 오류
296847에서 오류
296848에서 오류
296849에서 오류
296850에서 오류
296851에서 오류
296852에서 오류
296853에서 오류
296854에서 오류
296855에서 오류
296856에서 오류
296857에서 오류
296858에서 오류
296859에서 오류
296860에서 오류
296861에서 오류
296862에서 오류
296863에서 오류
296864에서 오류
296865에서 오류
296866에서 오류
296867에서 오류
296868에서 오류
296869에서 오류
296873에서 오류
296874에서 오류
296875에서 오류
296876에서 오류
296877에서 오류
296878에서 오류
296879에서 오류
296880에서 오류
296881에서 오류
296882에서 오류
296883에서 오류
296884에서 오류
296885에서 오류
296886에서 오류
296887에서 오류
296888에서 오류
296889에서 오류
296890에서 오류
296891에서 오류
296892에서 오류
296893에서 오류
296894에서 오류
296895에서 오류
296896에서 오류
296897에서 오류
296915에서 오류
296916에서 오류
296917에서 오류
296918에서 오류
296919에서 오류
296920에서 오류
296921에서 오류
296922에서 오류
296923에서 오류
296924에서 오류
296925에서 오류
296926에서 오류
296927에서 오류
296928에서 오류
296930에서 오류
296931에서 오류
296932에서 오류
296933에서 오류
296934에서 오류
296935에서 오류
296936에서 오류
296937에서 오류
296938에서 오류
296939에서 오류
296940에서 오류
296941에서 오류
296942에서 오류
296943에서 오류
296944에서 오류
296948에서 오류
296949에서 오류
296950에서 오류
296951에서 오류
296952에서 오류
2969

297955에서 오류
297956에서 오류
297957에서 오류
297958에서 오류
297959에서 오류
297960에서 오류
297961에서 오류
297962에서 오류
297963에서 오류
297964에서 오류
297965에서 오류
297966에서 오류
297967에서 오류
297968에서 오류
297969에서 오류
297970에서 오류
297971에서 오류
297972에서 오류
297973에서 오류
297974에서 오류
297975에서 오류
297976에서 오류
297977에서 오류
297978에서 오류
297979에서 오류
297980에서 오류
297981에서 오류
297982에서 오류
297983에서 오류
297984에서 오류
297985에서 오류
297986에서 오류
297987에서 오류
297988에서 오류
297989에서 오류
297990에서 오류
297991에서 오류
297992에서 오류
297993에서 오류
297994에서 오류
297995에서 오류
297996에서 오류
297997에서 오류
297998에서 오류
297999에서 오류
298000에서 오류
298001에서 오류
298002에서 오류
298003에서 오류
298004에서 오류
298005에서 오류
298006에서 오류
298007에서 오류
298008에서 오류
298009에서 오류
298010에서 오류
298011에서 오류
298012에서 오류
298013에서 오류
298014에서 오류
298015에서 오류
298016에서 오류
298017에서 오류
298018에서 오류
298019에서 오류
298020에서 오류
298021에서 오류
298022에서 오류
298023에서 오류
298024에서 오류
298025에서 오류
298026에서 오류
298027에서 오류
298028에서 오류
298029에서 오류
298030에서 오류
298031에서 오류
298032에서 오류
298033에서 오류
298035에서 오류
298037에서 오류
298055에서 오류
298056에서 오류
2980

298864에서 오류
298865에서 오류
298866에서 오류
298867에서 오류
298868에서 오류
298869에서 오류
298870에서 오류
298871에서 오류
298872에서 오류
298874에서 오류
298875에서 오류
298876에서 오류
298877에서 오류
298878에서 오류
298879에서 오류
298880에서 오류
298881에서 오류
298882에서 오류
298883에서 오류
298884에서 오류
298885에서 오류
298886에서 오류
298887에서 오류
298888에서 오류
298889에서 오류
298890에서 오류
298891에서 오류
298892에서 오류
298893에서 오류
298894에서 오류
298895에서 오류
298896에서 오류
298897에서 오류
298898에서 오류
298899에서 오류
298900에서 오류
298901에서 오류
298902에서 오류
298903에서 오류
298904에서 오류
298923에서 오류
298924에서 오류
298925에서 오류
298926에서 오류
298927에서 오류
298928에서 오류
298929에서 오류
298930에서 오류
298931에서 오류
298933에서 오류
298934에서 오류
298935에서 오류
298936에서 오류
298939에서 오류
298940에서 오류
298941에서 오류
298942에서 오류
298943에서 오류
298944에서 오류
298945에서 오류
298946에서 오류
298947에서 오류
298948에서 오류
298949에서 오류
298950에서 오류
298951에서 오류
298952에서 오류
298953에서 오류
298971에서 오류
298972에서 오류
298973에서 오류
298974에서 오류
298975에서 오류
298976에서 오류
298977에서 오류
298978에서 오류
298979에서 오류
298980에서 오류
298981에서 오류
298982에서 오류
298983에서 오류
298984에서 오류
298985에서 오류
2990

300558에서 오류
300559에서 오류
300560에서 오류
300561에서 오류
300562에서 오류
300563에서 오류
300564에서 오류
300565에서 오류
300566에서 오류
300567에서 오류
300569에서 오류
300570에서 오류
300571에서 오류
300572에서 오류
300574에서 오류
300575에서 오류
300576에서 오류
300577에서 오류
300578에서 오류
300579에서 오류
300580에서 오류
300581에서 오류
300582에서 오류
300583에서 오류
300584에서 오류
300585에서 오류
300586에서 오류
300587에서 오류
300588에서 오류
300589에서 오류
300590에서 오류
300591에서 오류
300592에서 오류
300593에서 오류
300594에서 오류
300595에서 오류
300596에서 오류
300597에서 오류
300598에서 오류
300599에서 오류
300600에서 오류
300601에서 오류
300602에서 오류
300603에서 오류
300604에서 오류
300605에서 오류
300606에서 오류
300607에서 오류
300608에서 오류
300609에서 오류
300610에서 오류
300611에서 오류
300612에서 오류
300613에서 오류
300614에서 오류
300615에서 오류
300616에서 오류
300617에서 오류
300618에서 오류
300619에서 오류
300620에서 오류
300621에서 오류
300622에서 오류
300623에서 오류
300624에서 오류
300625에서 오류
300626에서 오류
300627에서 오류
300628에서 오류
300629에서 오류
300713에서 오류
300714에서 오류
300715에서 오류
300716에서 오류
300717에서 오류
300718에서 오류
300719에서 오류
300720에서 오류
300721에서 오류
300722에서 오류
300723에서 오류
300724에서 오류
300726에서 오류
3007

...

1770798에서 오류
1770799에서 오류
1770800에서 오류
1770801에서 오류
1770802에서 오류
1770804에서 오류
1770805에서 오류
1770807에서 오류
1770809에서 오류
1770810에서 오류
1770811에서 오류
1770813에서 오류
1770814에서 오류
1770823에서 오류
1770824에서 오류
1770827에서 오류
1770895에서 오류
1770896에서 오류
1770897에서 오류
1770898에서 오류
1770899에서 오류
1770900에서 오류
1770901에서 오류
1770903에서 오류
1770904에서 오류
1770906에서 오류
1770907에서 오류
1770908에서 오류
1770909에서 오류
1770910에서 오류
1770912에서 오류
1770913에서 오류
1770916에서 오류
1770917에서 오류
1770918에서 오류
1770920에서 오류
1770921에서 오류
1770922에서 오류
1770924에서 오류
1770926에서 오류
1770928에서 오류
1770932에서 오류
1770935에서 오류
1770938에서 오류
1770939에서 오류
1770940에서 오류
1770945에서 오류
1770946에서 오류
1770948에서 오류
1770949에서 오류
1770950에서 오류
1770951에서 오류
1770955에서 오류
1770957에서 오류
1770958에서 오류
1770962에서 오류
1770964에서 오류
1770965에서 오류
1770966에서 오류
1770973에서 오류
1770982에서 오류
1770988에서 오류
1770989에서 오류
1770993에서 오류
1770996에서 오류
1770997에서 오류
1770998에서 오류
1770999에서 오류
1771000에서 오류
1771001에서 오류
1771002에서 오류
1771003에서 오류
1771004에서 오류
1771009에서 오류
1771010에서 오류
1771011에서 오류
1771012에서 오류

1773420에서 오류
1773421에서 오류
1773422에서 오류
1773423에서 오류
1773424에서 오류
1773425에서 오류
1773426에서 오류
1773428에서 오류
1773432에서 오류
1773435에서 오류
1773436에서 오류
1773437에서 오류
1773438에서 오류
1773439에서 오류
1773440에서 오류
1773441에서 오류
1773443에서 오류
1773444에서 오류
1773445에서 오류
1773451에서 오류
1773457에서 오류
1773458에서 오류
1773459에서 오류
1773463에서 오류
1773471에서 오류
1773476에서 오류
1773477에서 오류
1773478에서 오류
1773536에서 오류
1773544에서 오류
1773554에서 오류
1773718에서 오류
1773720에서 오류
1773724에서 오류
1773725에서 오류
1773726에서 오류
1773727에서 오류
1773728에서 오류
1773730에서 오류
1773734에서 오류
1773736에서 오류
1773738에서 오류
1773741에서 오류
1773742에서 오류
1773743에서 오류
1773744에서 오류
1773745에서 오류
1773746에서 오류
1773747에서 오류
1773748에서 오류
1773752에서 오류
1773753에서 오류
1773754에서 오류
1773755에서 오류
1773756에서 오류
1773757에서 오류
1773758에서 오류
1773761에서 오류
1773762에서 오류
1773764에서 오류
1773765에서 오류
1773767에서 오류
1773800에서 오류
1773801에서 오류
1773802에서 오류
1773803에서 오류
1773804에서 오류
1773805에서 오류
1773806에서 오류
1773807에서 오류
1773808에서 오류
1773809에서 오류
1773810에서 오류
1773811에서 오류
1773814에서 오류
1773815에서 오류
1773874에서 오류

1778570에서 오류
1778571에서 오류
1778572에서 오류
1778573에서 오류
1778580에서 오류
1778587에서 오류
1778588에서 오류
1778591에서 오류
1778600에서 오류
1778605에서 오류
1778606에서 오류
1778607에서 오류
1778608에서 오류
1778609에서 오류
1778610에서 오류
1778613에서 오류
1778614에서 오류
1778618에서 오류
1778629에서 오류
1778635에서 오류
1778640에서 오류
1778645에서 오류
1778652에서 오류
1778655에서 오류
1778665에서 오류
1778671에서 오류
1778676에서 오류
1778686에서 오류
1778687에서 오류
1778693에서 오류
1778694에서 오류
1778698에서 오류
1778712에서 오류
1778730에서 오류
1778733에서 오류
1778738에서 오류
1778742에서 오류
1778743에서 오류
1778744에서 오류
1778745에서 오류
1778898에서 오류
1778899에서 오류
1778900에서 오류
1778901에서 오류
1778903에서 오류
1778904에서 오류
1778906에서 오류
1778909에서 오류
1778912에서 오류
1778914에서 오류
1778915에서 오류
1778917에서 오류
1778920에서 오류
1778921에서 오류
1778922에서 오류
1778923에서 오류
1778925에서 오류
1778930에서 오류
1778931에서 오류
1778932에서 오류
1778933에서 오류
1778936에서 오류
1778937에서 오류
1778938에서 오류
1778939에서 오류
1778943에서 오류
1778944에서 오류
1778945에서 오류
1778947에서 오류
1778949에서 오류
1778952에서 오류
1778954에서 오류
1778956에서 오류
1778958에서 오류
1778962에서 오류
1778965에서 오류
1778966에서 오류

1782803에서 오류
1782805에서 오류
1782807에서 오류
1782808에서 오류
1782809에서 오류
1782811에서 오류
1782812에서 오류
1782814에서 오류
1782815에서 오류
1782816에서 오류
1782817에서 오류
1782818에서 오류
1782819에서 오류
1782824에서 오류
1782825에서 오류
1782826에서 오류
1782827에서 오류
1782828에서 오류
1782829에서 오류
1782831에서 오류
1782832에서 오류
1782833에서 오류
1782837에서 오류
1782843에서 오류
1782845에서 오류
1782847에서 오류
1782849에서 오류
1782852에서 오류
1782854에서 오류
1782855에서 오류
1782856에서 오류
1782857에서 오류
1782858에서 오류
1782861에서 오류
1782862에서 오류
1782863에서 오류
1782864에서 오류
1782870에서 오류
1782872에서 오류
1782874에서 오류
1782875에서 오류
1782880에서 오류
1782881에서 오류
1782883에서 오류
1782884에서 오류
1782885에서 오류
1782887에서 오류
1782888에서 오류
1782907에서 오류
1782908에서 오류
1782909에서 오류
1782910에서 오류
1782911에서 오류
1782912에서 오류
1782913에서 오류
1782941에서 오류
1782947에서 오류
1782948에서 오류
1782984에서 오류
1782999에서 오류
1783157에서 오류
1783160에서 오류
1783165에서 오류
1783168에서 오류
1783169에서 오류
1783173에서 오류
1783175에서 오류
1783177에서 오류
1783178에서 오류
1783179에서 오류
1783183에서 오류
1783184에서 오류
1783185에서 오류
1783196에서 오류
1783197에서 오류
1783204에서 오류
1783206에서 오류

,INS_DATE,SHPR_ADDR_1,CNEE_ADDR_1,송화 -> 터미널,터미널 -> 수화
1,20210303,경기도,경상남도,514547,5390
2,20210303,경기도,경상남도,514547,5390
3,20210303,경기도,서울특별시,514547,513833
4,20210303,경기도,경기도,514547,513833
5,20210303,경기도,경기도,514547,513833
...,...,...,...,...,...
1786343,20210702,경기도,서울특별시,528659,402678
1786347,20210702,경기도,경기도,528659,402678
1786348,20210703,경기도,제주특별자치도,402678,None
1786349,20210704,경기도,대전광역시,302624,75620


## 이월되는 데이터 오류 처리

In [6]:
INS_DATE_03 = FC_order[FC_order["INS_DATE"] == 20210331].index
INS_DATE_04 = FC_order[FC_order["INS_DATE"] == 20210430].index
INS_DATE_05 = FC_order[FC_order["INS_DATE"] == 20210531].index
INS_DATE_06 = FC_order[FC_order["INS_DATE"] == 20210630].index
INS_DATE = [INS_DATE_03,INS_DATE_04,INS_DATE_05,INS_DATE_06]
Next_Month = [20210401,20210501,20210601,20210701]

In [7]:
for ind in range(4):
    for index in INS_DATE[ind]:
        for i in list(loc_dict.keys())[:-1]:
            if FC_order["CNEE_ADDR_1"][index] in loc_dict.get(i):
                try:
                    FC_order["터미널 -> 수화"][index] = list(trafic[(trafic["집계일자"] == Next_Month[ind]) &\
                                                       (trafic["출발권역명"] == "수도권본부")][i[:-2]+"도착"])[0]
                except:
                    print("{}에서 오류".format(index))
                    FC_order["터미널 -> 수화"][index] = None

,INS_DATE,SHPR_ADDR_1,CNEE_ADDR_1,송화 -> 터미널,터미널 -> 수화
1,20210303,경기도,경상남도,514547,5390
2,20210303,경기도,경상남도,514547,5390
3,20210303,경기도,서울특별시,514547,513833
4,20210303,경기도,경기도,514547,513833
5,20210303,경기도,경기도,514547,513833
...,...,...,...,...,...
1786343,20210702,경기도,서울특별시,528659,402678
1786347,20210702,경기도,경기도,528659,402678
1786348,20210703,경기도,제주특별자치도,402678,None
1786349,20210704,경기도,대전광역시,302624,75620


## 제주도 예외 처리

In [8]:
Jeju_index = [FC_order[FC_order["CNEE_ADDR_1"] == "제주특별자치도"].index,FC_order[FC_order["CNEE_ADDR_1"] == "제주"].index]
for i in Jeju_index:
    for j in i:
        FC_order["송화 -> 터미널"][j] = "제주"
        FC_order["터미널 -> 수화"][j] = "제주"
FC_order

,INS_DATE,SHPR_ADDR_1,CNEE_ADDR_1,송화 -> 터미널,터미널 -> 수화
1,20210303,경기도,경상남도,514547,5390
2,20210303,경기도,경상남도,514547,5390
3,20210303,경기도,서울특별시,514547,513833
4,20210303,경기도,경기도,514547,513833
5,20210303,경기도,경기도,514547,513833
...,...,...,...,...,...
1786343,20210702,경기도,서울특별시,528659,402678
1786347,20210702,경기도,경기도,528659,402678
1786348,20210703,경기도,제주특별자치도,제주,제주
1786349,20210704,경기도,대전광역시,302624,75620


# 파일 저장

In [9]:
with open("FC_order_distance_trafic_end22.pickle","wb") as fw:
    pickle.dump(FC_order, fw)

0